# YOLO - Classificação de imagens (Upload)

Carregue uma ou várias imagens e obtenha as classes detectadas usando YOLO (OpenCV DNN).

Observações:
- Caso variáveis no .env não estejam definidas, um modelo YOLOv3-tiny COCO será baixado automaticamente.
- As classes do dataset custom (car, motorbike, threewheel, van, bus, truck) aparecerão quando o modelo treinado for usado.

In [1]:
# Importa dependências e inicializa o detector (env ou fallback)
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FileUpload, Button, VBox, HBox, Output
from yolo_inference import build_detector_from_env

detector = build_detector_from_env()
out = Output()  # Saída para logs e resultados

In [2]:
# Widget de upload e botão para executar a detecção
uploader = FileUpload(accept='image/*', multiple=True)
btn = Button(description='Detectar', button_style='success')

def process_images(_):
    # Executa detecção para cada imagem anexada e exibe resultados
    out.clear_output()
    with out:
        if not uploader.value:
            print('Nenhuma imagem anexada')
            return
        for name, item in uploader.value.items():
            data = np.frombuffer(item['content'], dtype=np.uint8)
            img = cv2.imdecode(data, cv2.IMREAD_COLOR)
            if img is None:
                print(f'Falha ao carregar {name}')
                continue
            detections = detector.detect(img)
            img_out = detector.draw(img, detections)
            img_rgb = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 8))
            plt.imshow(img_rgb)
            plt.axis('off')
            if detections:
                print(f'{name}:')
                for d in detections:
                    print(f" - {d['class_name']} ({d['confidence']:.2f})")
            else:
                print(f'{name}: nenhuma detecção acima do limiar')

btn.on_click(process_images)
VBox([HBox([uploader, btn]), out])